# BLU12 - Learning Notebook- Part 3 of 3 - Advanced Topics

In [ ]:
import os

from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import KNNBasic

from scipy.sparse import coo_matrix, dok_matrix

from make_data import make_data
from export_ratings import export_ratings

# 1 Implicit Feedback

Most times, RS algorithms ingest implicit feedback, i.e., unary ratings, to understand user preferences.

In such cases, the unary data indicates whether a user $u \in U$ performed a given action (e.g., click, purchase).

Afterward, this data is used on its own or combined with explicit ratings.

In a way, ratings from unary data are ratings $r_{ui} \in S = \{1\}$, i.e., with a singleton or unit set of possible ratings $S$.

Absent ratings $r_ui \notin R$ indicates that we have no information relating the user $u$ to the item $i$, just like before.

(Perhaps the user purchased the item somewhere else, or the user didn't click the item because he didn't see it.)

![collaborative_filtering_unary](../media/collaborative_filtering_unary.png)

We make, however, some distinctions. 

## 1.1 Example

We generated some fake unary data, using the [Faker](https://faker.readthedocs.io/en/master/) package.

In `Learning Notebooks/make_data.py`, you find the `make_data()` function that generates two COO sparse matrices. 

This function is exactly like in the learning materials and exercises from [BLU11](https://github.com/LDSSA/batch2-blu11), so we don't repeat it.

In [ ]:
users, items, clicks, purchases = make_data()

In [ ]:
clicks

In [ ]:
purchases

The data contains exactly 50 users and 50 items, i.e., $|U| = 50$ and $|I| = 50$.

We include 500 clicks and 500 purchases for us to play with.

## 1.2 Duplicate Entries

For starters, the user $u \in U$ can perform an action, i.e., implicitly rate, multiple times for the same item $i$.

This violates the assumptions of the matrix $R$, so upstream consolidation is required, enforcing one rating $r_ui$ for each pair $(u, i) \in U \times I$.

Again, let $A$ be set of unary ratings, i.e., $a_{ui} \in S = \{1\}$, for user-item pairs $(u, i) \in U \times I$, which contains duplicate pairs.

A common technique is to sum together duplicate entries, as:

$$\sum\limits_{(u, i) \in U \times I} a_{ui}$$

As we've seen in [BLU11](https://github.com/LDSSA/batch2-blu11), this is the default behavior when we convert from COO to CSR.

In [ ]:
clicks_ = clicks.tocsr()
clicks_

The reduction from 500 to 460 stored element in the matrix is due to the consolidation.

We can confirm this by calling `.max()` on it.

In [ ]:
clicks_.max()

In [ ]:
purchases_ = purchases.tocsr()

In [ ]:
purchases_

In [ ]:
purchases_.max()

Another conventional technique is to use the logarithm of the sum, instead.

$$\log{\sum\limits_{(u, i) \in U \times I} a_{ui}}$$

The log transformation is particularly useful with right-skewed distributions, i.e., not centered, with a peak on the left and a tail on the right.

(Imagine a user $u$ with few clicks on many items and many of clicks on a few items, which is very common.)

We can apply this quickly if so we choose, by applying the logaritm element-wise on the resulting matrix.

In [ ]:
clicks_.log1p()

In [ ]:
purchases.log1p()

## 1.3 Inferring Ratings

Also, since we have multiple signals relating the user $u$ to item $i$, we have to consolidate them into a single rating.

Different signals (e.g., impressions, clicks, purchases) have distinct signal-to-noise ratios and levels of intent and, thus, may require different weights.

Consider the set $D$, containing all types of implicit feedback, e.g., $D = \{Click, Purchase\}$, with the associated weights $W$.

We can compute the ratings $r_{ui}$, for $(u, i) \in U \times I$, as:

$$r_{ui} = \sum\limits_{(u, i) \in U \times I} \Big(\sum\limits_{d \in D} w_d \cdot a_{ui}^d \Big)$$

In our example, we attribute more relevance to purchases than clicks.

(Please note that Python silently converts from COO to CSR, summing together duplicate entries by default.)

In [ ]:
def make_ratings(c, p, w_c, w_p):
    return w_c * c + w_p * p 


ratings = make_ratings(clicks, purchases, .3, .7)
ratings

## 1.4 Exporting Ratings

Once we have final ratings, it's good practice to export them in long-form, using the `'uid,iid,rating'` convention.

We can do this easily, by converting back to COO and use the `.row`, `.col` and `.data` attributes.

In [ ]:
ratings_ = ratings.tocoo()

uid = np.array([users[row] for row in ratings_.row], dtype='O')
iid = np.array([items[col] for col in ratings_.col], dtype='O')

In [ ]:
data = ratings_.data

For full implementation detail and NumPy nitty gritty, refer to `Learning Notebooks/export_ratings.py`.

In [ ]:
export_ratings(users, items, ratings)

From here onwards, we can use all the RS techniques we have learned.

(Including using the Surprise package.)

# 2 Generating top-*N* Lists

Often, we task the RS with recommending a list $L_u$, containing $N$ items likely to be of interest to an active user $u$.

This type of output is particularly frequent in the presence of implicit feedback and unary data, as ratings loose meaning *per se*.

How can we generate such a list $L_u$, using Surprise?

In [ ]:
dataset = Dataset.load_builtin('ml-100k')
R_train = dataset.build_full_trainset()

We will use the `KNNBasic` to generate predictions, with all the defaults.

(This may take a few minutes.)

In [ ]:
knn = KNNBasic()
knn.fit(R_train)

R_test = R_train.build_anti_testset()
R_pred = knn.test(R_test)

From the Surprise documentation, [this](https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user) is the recommended way to extract a top-$N$ list for each user. 

(Slightly adapted, so that we can use it in the future).

In [ ]:
def get_top_n(predictions, n=10):
    
    top_n = defaultdict(list)
    
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = [x[0] for x in user_ratings[:n]]

    return pd.DataFrame.from_dict(data=top_n, orient='index')


L = get_top_n(R_pred, n=10)
L.head()

This way, we generate a ranked list of recommendations $L_u$ for each user $u \in U$, in a convenient format:
* One row per user, indexed with the `uid`
* One column per recommendation, ordered by the estimated ranking.

Now, we learn how to evaluate algorithms focused on learning top-$N$ lists.

# 3 Evaluation Metrics for top-*N* Lists

When ratings are not available, i.e., with unary data, measuring the rating prediction accuracy isn't possible.

In these cases, evaluation is done using $R_{train}$ to learn $L_u$ and evaluating on $R_{test}$

Let $T_u \subset I_u \cap I_{test}$ the subset of test items that the user $u$ found relevant, e.g., rated positively, clicked, purchased.

## 3.1 Precision

Precision measures how many recommended items are relevant, out of all recommended items to the user $u$.

$$Precision(L_u) = \frac{|L_u \cap T_u |}{|L_u|}$$

To evaluate the RS as a whole, we average the precision for all active users $u \in U$.

$$Precision(L) = \frac{\sum\limits_{u \in U} Precision(L_u)}{|U|}$$

## 3.2 Recall

Recall, on the other side, relates to how many relevant were recommended, out of all relevant items for the user $u$.

$$Recall(L_u) = \frac{|L_u \cap T_u |}{|T_u|}$$

Again, to evaluate the TS we average the results of all active users $u \in U$.

$$Recall(L) = \frac{\sum\limits_{u \in U} Recall(L_u)}{|U|}$$

## 3.3 Average Precision (AP)

Precision and recall ignore the ordering. Therefore we need a ranking metric.

To understand average precision, we must start with Precision@k and Recall@k, i.e., precision and recall up to cut-off $k$.

In other words, we consider only the subset of recommendations $L_u^k \subset L_u$ from rank 1 through rank $k \leqslant N$.

$$PrecisionAtk(L_u) = \frac{|L_u^k \cap T_u |}{|L_u^k|}$$

$$RecallAtk(L_u) = \frac{|L_u^k \cap T_u |}{|T_u|}$$

The AP is a ranking metric, measuring the frequency of relevant recommendations.

$$APatN(L_u) = \frac{\sum\limits_{k = 1}^N (PrecisionAtk(L_u) \cdot relevant(k^{th})}{|T_u|}$$

The $relevant(k^{th})$ bit is a boolean value, indicating whether the $k$-th element is relevant, or not.

Every hit is valued as how many correct recommendations $|L_u^k \cap T_u|$ we have up to the rank $k$, out of all recommendations $|L_u^k|$.

A first interpretation is that the AP increases only with correct recommendations (what a surprise!).

Also, early hits, i.e., front-loading correct recommendations, carry over and are continuously rewarded.

Finally, the AP can never decrease as you increase $N$.

There is, however, an alternative formula for AP, in terms of both precision and the change in recall from the subset $k$ − 1 to the $k$-th.

$$APatN(L_u) = \sum\limits_{k=1}^NPrecisionAtk(L_u) * \Delta RecallAtk(L_u)$$ 

## 3.4 Mean Average Precision (mAP)

The Average Precision (AP) is further averaged over all users and reported as a single score.

$$mAPatN(L) = \frac{\sum\limits_{u \in U} APatN(L_u)}{|U|}$$

This way, we use a metric that considers both the number and the ranking of hits, i.e., useful recommendations.

In this last section, we learned how to use unary data, make predictions based on it and how to evaluate our algorithms.

Time to practice!